In [1]:
import pandas
import datetime
import os
import sys
module_path = os.path.abspath(os.path.join('../data_pipeline/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import sqlite3

/usr/local/lib/python2.7/dist-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/dist-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/dist-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/usr/local/lib/python2.7/dist-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/usr/local/lib/python2.7/dist-packages/pandas/core/indexes/base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate bi

In [2]:
ae_dbdir = "/home/bharat/Documents/data/ss_onset_dataset/data/sqlite3/"
ae_db_name = "au_al_ae.sqlite"
omn_table_name = "aualae"
start_date = datetime.datetime(1996,1,1)
end_date = datetime.datetime(1996,12,31)
# set some cutoffs for AL/AE indices
alSSCutoff = -10
aeSSCutoff = 50
# set some cutoffs for time
minDelT = 5 # min
# the continuous set of time range
# which is considered a non-substorm
# period. In other words, if non-SS
# conditions are satisfied for this
# interval of time (say 3 hours), then
# we consider this period to be a non-ss
# period.
minDiffTime = 180 # min

In [3]:
conn = sqlite3.connect(ae_dbdir + ae_db_name,
                       detect_types = sqlite3.PARSE_DECLTYPES)
# load data to a dataframe
command = "SELECT * FROM {tb} " +\
                  "WHERE datetime BETWEEN '{stm}' and '{etm}'"
command = command.format(tb=omn_table_name,\
                         stm=start_date, etm=end_date)
aulDF = pandas.read_sql(command, conn)

In [4]:
aulDF = aulDF[ (aulDF["al"] >= alSSCutoff) &\
             (aulDF["ae"] <= aeSSCutoff)\
             ].reset_index(drop=True)
# Calculate the time diff between two consecutive timesteps
aulDF["delT"] = aulDF["datetime"].diff()
# convert the difference to minutes
aulDF["delT"] = aulDF["delT"].apply(\
            lambda x: x.total_seconds()/60. )

In [5]:
print aulDF[ aulDF["delT"] > 1. ].shape, aulDF[ aulDF["delT"] == 1. ].shape

(5007, 5) (56488, 5)


In [6]:
# get contiguous set of measurements where
# there is no ss
# here we're getting a diff on the series to get
# the most continuous dataset
brkInds = aulDF[ (aulDF["delT"] > minDelT)\
               ].index.to_frame().diff().reset_index()
brkInds.columns = [ "inds", "diffs" ]
# now we also need index value from prev row
# to get the range of time
shftdRows = brkInds["inds"].shift(1)
brkInds["prevRowInds"] = shftdRows

In [7]:
# the continous set of time with no substorm activity 
# would be [prevRowInds:inds-1]
print brkInds[ brkInds["diffs"] > minDiffTime ].head(10)

     inds  diffs  prevRowInds
2     495  360.0        135.0
16    919  202.0        717.0
117  3138  217.0       2921.0
123  3424  261.0       3163.0
126  4039  599.0       3440.0
144  4767  299.0       4468.0
248  6647  286.0       6361.0
252  6916  256.0       6660.0
353  8802  506.0       8296.0
357  9058  228.0       8830.0


In [10]:
for row in brkInds[ brkInds["diffs"] > minDiffTime ].iterrows():
    dd = aulDF.iloc[ int(row[1]["prevRowInds"]): int(row[1]["inds"]-1)]["datetime"]
    print dd.min(), dd.max()

1996-01-01 02:38:00 1996-01-01 08:44:00
